# Tutorial on GW Parameter Inference with Machine Learning 

## Kavli-Villum Summer School on Gravitational Waves

### Stephen Green *stephen.green2@nottingham.ac.uk*

---

In this tutorial we will build a simple neural network to do **parameter estimation** using simulation-based inference.

### Steps

1. **Build a set of training waveforms.** We use the IMRPhenomPv2 model, parametrized only by masses.
2. **Build a posterior model $q(\theta | d)$** using a neural network. We consider two cases:
    - Gaussian distribution with diagonal (learnable) covariance.
    - Normalizing flow (masked autoregressive flow).
3. **Train the model to represent the posterior**, i.e., $q(\theta|d) \to p(\theta|d)$.
    - During training, we add noise to waveforms to make simulated data.
4. **Evaluate** on test data.

This should run in a few minutes on a laptop.

### Exercises

1. Add new parameters, beyond the masses
2. Use a more complicated waveform model
3. Make a PP plot
4. Experiment with the spline flow

## Imports

We use the [PyTorch](https://pytorch.org) ML library. Waveforms are generated using `lalsimulation`.

In [ ]:
# If using colab, install appropriate packages.
!pip install corner
!pip install lalsuite

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import lal
import lalsimulation as LS
import corner

In [ ]:
# PyTorch

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
from torch.utils.data import DataLoader

In [ ]:
# If running on Apple Silicon or CUDA is available, use the GPU.

if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

## Training data

### Signal model

Generic quasi-circular inspirals depend on 15 parameters $\theta$:
* Masses (2 parameters)
* Spins (6 parameters)
* Inclination
* Sky position (2 parameters)
* Luminosity distance
* Time of arrival
* Polarization
* Phase

We can write $h_I(\theta)$ for the signal in detector $I$.

However, for this tutorial we will restrict consideration to only the masses, fixing everything else. Rather than considering waveforms in detectors, we will also only analyze the plus polarization.

**Exercise:** Perform inference for more parameters, e.g., luminosity distance, inclination, phase, or aligned spins. (For sky position, it is necessary to project the polarizations onto detectors.)

### Noise

Ultimately we must train on simulated data, which also include noise,
$$
d = h(\theta) + n, \qquad n \sim p_{S_n}(n).
$$
The noise is taken to be stationary Gaussian with some power spectral density $S_n$.

Rather than creating complete simulated data sets in advance of training, **we only prepare the waveforms in advance, and we add noise realizations during training.** The reason for this is that we would like the training dataset to be as large as possible to reduce the risk of overfitting. Noise is fast to sample, so this can be done during training, and doing so effectively makes the training set much larger. Generally waveforms are slower to generate, so we re-use them in each epoch.

In [ ]:
# Size of the training set.
num_samples = 10000

# Sample the masses over a uniform prior.
m_lower = 20.0  # solar masses
m_upper = 80.0
masses = m_lower + np.random.random((num_samples, 2)) * (m_upper - m_lower)

# Make sure m1 > m2
masses = np.sort(masses, axis=-1)
masses = np.flip(masses, axis=-1)

In [ ]:
# Fixed parameters

distance = 1000  # Mpc
inclination = 0.0  # face on
spin1x = 0.0
spin1y = 0.0
spin1z = 0.0
spin2x = 0.0
spin2y = 0.0
spin2z = 0.0
phase = 0.0

In [ ]:
# Waveform settings

approximant = 'IMRPhenomPv2'
f_min = 20.0
f_max = 512.0
T = 2.0

delta_f = 1 / T
nf = int(f_max / delta_f) + 1
f_array = np.linspace(0.0, f_max, num=nf)

In [ ]:
# Generate the noise PSD. We take this to be the LIGO design sensitivity.

lalseries = lal.CreateREAL8FrequencySeries('', lal.LIGOTimeGPS(0), 0, delta_f, lal.DimensionlessUnit, nf)
LS.SimNoisePSD(lalseries, 0, LS.SimNoisePSDaLIGOZeroDetHighPowerPtr)
psd = lalseries.data.data
psd[:int(f_min/delta_f)] = 1
psd[-1] = psd[-2]

asd = np.sqrt(psd)

In [ ]:
# Plot the ASD.

plt.plot(f_array, asd)
plt.xlim((f_min, f_max))
plt.ylim((3e-24, 3e-23))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('f [Hz]')
plt.ylabel('ASD')
plt.show()

In [ ]:
# Generate training waveforms

hp_list = []
hc_list = []

for i in range(num_samples):
    
    mass1, mass2 = masses[i]

    mass1_lal = mass1 * lal.MSUN_SI
    mass2_lal = mass2 * lal.MSUN_SI
    distance_lal = distance * 1e6 * lal.PC_SI
    approximant_lal = LS.GetApproximantFromString(approximant)

    hp, hc = LS.SimInspiralFD(
        mass1_lal, mass2_lal,
        spin1x, spin1y, spin1z, spin2x, spin2y, spin2z, 
        distance_lal, inclination, phase,
        0.0, 0.0, 0.0,  # longAscNodes, eccentricity, meanPerAno
        delta_f, f_min, f_max, 20.0,  # f_ref
        None,
        approximant_lal,
    )

    dt = 1 / hp.deltaF + (hp.epoch.gpsSeconds + hp.epoch.gpsNanoSeconds * 1e-9)
    time_shift = np.exp(-1j * 2 * np.pi * dt * f_array)
    
    hp = hp.data.data
    hc = hc.data.data

    hp *= time_shift
    hc *= time_shift
    
    hp[:int(f_min/delta_f)] = 0.0
    hc[:int(f_min/delta_f)] = 0.0
    
    # Whiten waveforms and rescale so that white noise has unit variance in each frequency bin.
    # This makes it easy to add noise, and also normalized data works better as input to neural networks.
    
    hp = hp / asd * np.sqrt(4.0 * delta_f)
    hc = hc / asd * np.sqrt(4.0 * delta_f)
    
    hp_list.append(hp)
    hc_list.append(hc)

hp = np.array(hp_list)
hc = np.array(hc_list)

In [ ]:
# Sample waveform

plt.plot(f_array, hp[0].real)
plt.xscale('log')
plt.xlabel('$f$')
plt.ylabel('Re $h_+$')
plt.xlim((10, f_max))
plt.show()

### Package into a PyTorch Dataset

The `Dataset` is a convenient class for storing and accessing pairs of parameters and associated data. It must define the following methods:

* `__len__()`: Return total number of samples in the dataset.
* `__getitem__(idx)`: Retrieve a $(\theta, d)$ pair of parameters and data. We use this method to also add (in real time) a noise realization to each simulated waveform. (Therefore repeated calls will give different noise realizations).

For now, we will only make use of the plus polarization for simplicity.

**Exercise:** Extend this to include more parameters. Some parameters, e.g., luminosity distance, can be easily transformed. For these, it makes sense to select them in real time, like for the noise.

In [ ]:
# Parameters
#
# This is just the masses. It's better to sample in (Mc, q) rather than (m1, m2) because the posterior is more Gaussian.

m1 = masses[:, 0]
m2 = masses[:, 1]

Mc = (m1 * m2)**(3/5) / (m1 + m2)**(1/5)
q = m2 / m1

parameters = np.stack((Mc, q), axis=1).astype(np.float32)

In [ ]:
# For best training, parameters should be standardized (zero mean, unit variance across the training set)

parameters_mean = np.mean(parameters, axis=0)
parameters_std = np.std(parameters, axis=0)

parameters_standardized = (parameters - parameters_mean) / parameters_std

In [ ]:
# Waveforms
#
# Truncate the arrays to remove zeros below f_min, and repackage real and imaginary parts
#
# Only consider h_plus for now

lower_cut = int(f_min / delta_f)
waveforms = np.hstack((hp.real[:, lower_cut:], hp.imag[:, lower_cut:])).astype(np.float32)

In [ ]:
class WaveformDataset(Dataset):
    
    def __init__(self, parameters, waveforms):
        self.parameters = parameters
        self.waveforms = waveforms

    def __len__(self):
        return len(self.parameters)

    def __getitem__(self, idx):
        params = self.parameters[idx]
        signal = self.waveforms[idx]
        
        # Add unit normal noise to the signal
        noise = np.random.normal(size = signal.shape).astype(np.float32)
        data = signal + noise
        
        return torch.tensor(data, device=device), torch.tensor(params, device=device)

In [ ]:
waveform_dataset = WaveformDataset(parameters_standardized, waveforms)

In [ ]:
# We can sample from the WaveformDataset. This gives us pairs of data and parameters, different noise realizations each time.

x, y = waveform_dataset[0]
x = x.cpu()  # Careful to put data back onto CPU.
plt.plot(x)
plt.show()

## Gaussian posterior model

We now try to fit the posterior using a Gaussian model, i.e.,

$$
q(\theta|d) = \mathcal{N}(\mu(d), \Sigma(d)) (\theta).
$$
Here $\mu$ and $\Sigma$ are the mean and covariance matrix describing the Gaussian, and they are given as the output of a neural network, which takes as input the data $d$. Additionally, we constrain $\Sigma$ to be diagonal, so we only learn the variance in each parameter, and we do not allow for correlations between parameters in the posterior. (**Exercise:** Lift this restriction.)

### Model construction

Neural networks are constructed by subclassing `nn.Module`.

This has to implement an `__init__()` and `forward()` method.

In [ ]:
# Neural networks are constructed by subclassing nn.Module
#
# This has to implement an __init__() and forward() method

class GaussianNeuralNetwork(nn.Module):
    
    def __init__(self, input_dim, hidden_dims, output_dim, activation=nn.ReLU()):
        super(NeuralNetwork, self).__init__()
        
        # Hidden layers
        hidden_net_list = []
        hidden_net_list.append(
            nn.Linear(input_dim, hidden_dims[0]))
        for i in range(1, len(hidden_dims)):
            hidden_net_list.append(nn.Linear(hidden_dims[i-1], hidden_dims[i]))
        self.hidden_net_list = nn.ModuleList(hidden_net_list)
        
        # Output layers
        self.output_mean = nn.Linear(hidden_dims[-1], output_dim)
        self.output_log_sigma = nn.Linear(hidden_dims[-1], output_dim)
        
        # Activation function
        self.activation = activation
        
    def forward(self, x):
        """Pass x through all the layers of the network and return the Gaussian distribution"""
        
        h = x
        for layer in self.hidden_net_list:
            h = self.activation(layer(h))

        # Output layer defines a Gaussian
        mean = self.output_mean(h)
        log_sigma = self.output_log_sigma(h)
        sigma = torch.exp(log_sigma)
        
        # Create the Gaussian distribution
        dist = torch.distributions.MultivariateNormal(loc=mean, scale_tril=torch.diag_embed(sigma))
        
        return dist

In [ ]:
input_dim = waveforms.shape[-1]
output_dim = parameters.shape[-1]
hidden_dims = [512, 256, 128, 64, 32]

model = GaussianNeuralNetwork(input_dim, hidden_dims, output_dim)

model.to(device)  # Put model on GPU, if available.

### Training

In [ ]:
# Split the dataset into training and test sets.
# We use the test set to make sure the network properly generalizes to data that it has not seen in training, i.e., it does not overfit.

train_fraction = 0.8
num_train = int(round(train_fraction * num_samples))
num_test = num_samples - num_train
train_dataset, test_dataset = random_split(waveform_dataset, [num_train, num_test])

# The DataLoader is used to obtain samples form the Dataset during training.

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# The DataLoaders iterate over samples, returning torch tensors containing a batch of data

train_features, train_labels = next(iter(train_dataloader))

In [ ]:
train_features

In [ ]:
train_features.shape

In [ ]:
# We use the Adam optimizer.

optimizer = torch.optim.Adam(model.parameters())

In [ ]:
# Training and test loops

def train_loop(dataloader, model, optimizer):
 
    size = len(dataloader.dataset)
    train_loss = 0
    
    for batch, (X, y) in enumerate(dataloader):
        # Compute negative log probability loss
        dist = model(X)        
        loss = - dist.log_prob(y)
        
        train_loss += loss.detach().sum()
        loss = loss.mean()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 50 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Loss: {loss:>7f}  [{current:>5d}/{size:>5d} samples]")
            
    average_loss = train_loss.item() / size
    print('Average loss: {:.4f}'.format(average_loss))
    return average_loss
            
        
        
def test_loop(dataloader, model):
    size = len(dataloader.dataset)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            dist = model(X)
            loss = - dist.log_prob(y)
            test_loss += loss.sum()

    test_loss = test_loss.item() / size
    print(f"Test loss: {test_loss:>8f} \n")
    return test_loss

In [ ]:
# Train for 20 epochs. 

epochs = 20
train_history = []
test_history = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    loss = train_loop(train_dataloader, model, optimizer)
    train_history.append(loss)
    loss = test_loop(test_dataloader, model)
    test_history.append(loss)
print("Done!")

In [ ]:
epochs = np.arange(1, len(train_history) + 1)
plt.plot(epochs, train_history, label = 'train loss')
plt.plot(epochs, test_history, label = 'test loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

We see that the test loss drops just as fast as the train. Since these curves coincide we conclude that the model is not overfitting.

If we trained longer, or used a more powerful model, we might find a difference between these curves. If this happens, increase the size of the training set.

In [ ]:
num_posteriors = 5
num_samples = 10000

model.eval()

for n in range(num_posteriors):
    with torch.no_grad():
        test_x, test_y = test_dataset[n]
    
        # Predict a posterior
        dist = model(test_x)
    
        # Sample the posterior
        pred_samples = dist.sample((10000,)).cpu().numpy()

        # Undo the standardization
        pred_samples = parameters_std * pred_samples + parameters_mean
        truth = parameters_std * test_y.cpu().numpy() + parameters_mean

    # Plot
    corner.corner(pred_samples, truths=truth, labels=['$M_c$', '$q$'])
    plt.show()

In [ ]:
num_posteriors = 5
num_eval_samples = 10000

model.eval()

for n in range(num_posteriors):

    with torch.no_grad():
    
        test_x, test_y = test_dataset[n]
       
        # Sample the posterior
        test_x = test_x.expand(num_eval_samples, *x.shape)
        pred_samples = model.sample(1, test_x).squeeze(1).cpu().numpy()
    
        # Undo the standardization
        pred_samples = parameters_std * pred_samples + parameters_mean
        truth = parameters_std * test_y.cpu().numpy() + parameters_mean
    
        # Plot
        corner.corner(pred_samples, truths=truth, labels=['$M_c$', '$q$'])
        plt.show()

## Normalizing flow

Normalizing flows allow for much more general probability distributions, as needed to represent GW posterior distributions in 15D. A normalizing flow represents the distribution in terms of a mapping $f_d:\mathbb R^D \to \mathbb R^D$ from a *base distribution* $\pi(u) = \mathcal N(0,1)^D$,
$$
q(\theta | d) = \pi(f_d^{-1}(\theta)) | \det J_{f_d}^{-1} |.
$$
To sample $\theta \sim q(\theta|d)$, first sample $u\sim \pi(u)$, then apply the mapping and set $\theta = f_d(u)$. By having $f_d$ depend on the data $d$, this describes a conditional distribution, as needed for amortized inference.

To train the flow, it is necessary also to evaluate the density. This can be done by evaluating the right hand side of the equation above, provided two properties hold:
1. $f_d$ invertible
2. $f_d$ has simple Jacobian determinant
   
The trick then is to construct suitable $f_d$ such that these properties hold. A simple model is a [Masked Autoregressive Flow (MAF)](https://arxiv.org/abs/1705.07057).

### Model construction

Normalizing flows are constructed as a sequence of discrete steps, each of which is relatively simple. For an autoregressive flow step $n$, we write
$$
q^{(n)}(\theta | d) = \prod_{i=1}^D q^{(n)}(\theta_i | \theta_{1:i-1}, d),
$$
where each factor takes a particular parametrized form. For the MAF, this is taken to be a univariate normal distribution,
$$
q^{(n)}(\theta_i | \theta_{1:i-1}, d) = \mathcal{N}\left(\mu_i^{(n)}(\theta_{1:i-1},d), \sigma_i^{(n)}(\theta_{1:i-1},d)\right)
$$
so that the corresponding mapping is an affine transformation. The means and variances are given as output of a neural network, and are learned. There is in fact a smart way of representing all of these quantities for a single step in one neural network with masking called [MADE](https://arxiv.org/abs/1502.03509). Between each flow step we also randomize the order of the parameters.

For the normalizing flow, we use an off-the-shelf library rather than coding it ourselves. Although we will use a MAF for our example, code for a [spline flow](https://arxiv.org/abs/1906.04032) is also included. **Exercise:** Try it out.

In [ ]:
# The glasflow library is a wrapper for nflows (https://github.com/bayesiains/nflows), which implements many different flows.

from glasflow.nflows import transforms, distributions, flows
import glasflow.nflows as nflows
import glasflow.nflows.nn.nets as nflows_nets

In [ ]:
device = 'cpu'  # For some reason, "mps" does not work here.

In [ ]:
# Code here is adapted from examples in the neural spline flow repository, https://github.com/bayesiains/nsf

# These functions create the transform sequence.

def create_base_transform(
    i: int,
    param_dim: int,
    context_dim: int = None,
    hidden_dim: int = 512,
    num_transform_blocks: int = 2,
    batch_norm: bool = False,
    base_transform_type: str = "maf",
):
    
    activation_fn = nn.ELU()

    if base_transform_type == "maf":
        return transforms.MaskedAffineAutoregressiveTransform(
            param_dim,
            hidden_features=hidden_dim,
            context_features=context_dim,
            num_blocks=num_transform_blocks,
            activation=activation_fn,
            use_batch_norm=batch_norm,
        )

    elif base_transform_type == "rq-coupling":
        if param_dim == 1:
            mask = torch.tensor([1], dtype=torch.uint8)
        else:
            mask = nflows.utils.create_alternating_binary_mask(
                param_dim, even=(i % 2 == 0)
            )
        return transforms.PiecewiseRationalQuadraticCouplingTransform(
            mask=mask,
            transform_net_create_fn=(
                lambda in_features, out_features: nflows_nets.ResidualNet(
                    in_features=in_features,
                    out_features=out_features,
                    hidden_features=hidden_dim,
                    context_features=context_dim,
                    num_blocks=num_transform_blocks,
                    activation=activation_fn,
                    use_batch_norm=batch_norm,
                )
            ),
            num_bins=8,
            tails="linear",
            tail_bound=1.0,
            apply_unconditional_transform=False,
        )

    else:
        raise ValueError


def create_linear_transform(param_dim: int):
    return transforms.CompositeTransform(
        [
            transforms.RandomPermutation(features=param_dim),
            transforms.LULinear(param_dim, identity_init=True),
        ]
    )


def create_transform(
    num_flow_steps: int, param_dim: int, context_dim: int, base_transform_kwargs: dict
):
    return transforms.CompositeTransform(
        [
            transforms.CompositeTransform(
                [
                    create_linear_transform(param_dim),
                    create_base_transform(
                        i, param_dim, context_dim=context_dim, **base_transform_kwargs
                    ),
                ]
            )
            for i in range(num_flow_steps)
        ]
        + [create_linear_transform(param_dim)]
    )

In [ ]:
# The entire flow model consists of a standard normal base distribution, followed by the transform.

transform = create_transform(num_flow_steps = 3,
                             param_dim = parameters.shape[-1],
                             context_dim = waveforms.shape[-1],
                             base_transform_kwargs = {"base_transform_type": "maf"})

base_distribution = distributions.StandardNormal(shape=[parameters.shape[-1]])

model = flows.Flow(transform, base_distribution)

In [ ]:
model.to(device)

### Training

In [ ]:
# Split the dataset into training and test sets

train_fraction = 0.8
num_train = int(round(train_fraction * num_samples))
num_test = num_samples - num_train
train_dataset, test_dataset = random_split(waveform_dataset, [num_train, num_test])

# The DataLoader is used in training

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# We use the Adam optimizer.

optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)

In [ ]:
# Training and test loops

def train_loop(dataloader, model, optimizer):

    model.train()
 
    size = len(dataloader.dataset)
    train_loss = 0
    
    for batch, (X, y) in enumerate(dataloader):
        # Compute negative log probability loss  
        loss = - model.log_prob(y, X)
        
        train_loss += loss.detach().sum()
        loss = loss.mean()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 50 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Loss: {loss:>7f}  [{current:>5d}/{size:>5d} samples]")
            
    average_loss = train_loss.item() / size
    print('Average loss: {:.4f}'.format(average_loss))
    return average_loss
            
        
        
def test_loop(dataloader, model):

    model.eval()
    
    size = len(dataloader.dataset)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            loss = - model.log_prob(y, X)
            test_loss += loss.sum()

    test_loss = test_loss.item() / size
    print(f"Test loss: {test_loss:>8f} \n")
    return test_loss

In [ ]:
epochs = 20
train_history = []
test_history = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    loss = train_loop(train_dataloader, model, optimizer)
    train_history.append(loss)
    loss = test_loop(test_dataloader, model)
    test_history.append(loss)
print("Done!")

In [ ]:
epochs = np.arange(1, len(train_history) + 1)
plt.plot(epochs, train_history, label = 'train loss')
plt.plot(epochs, test_history, label = 'test loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Evaluation

In [ ]:
num_posteriors = 5
num_eval_samples = 10000

model.eval()

for n in range(num_posteriors):

    with torch.no_grad():
    
        test_x, test_y = test_dataset[n]
       
        # Sample the posterior
        test_x = test_x.expand(num_eval_samples, *x.shape)
        pred_samples = model.sample(1, test_x).squeeze(1).cpu().numpy()
    
        # Undo the standardization
        pred_samples = parameters_std * pred_samples + parameters_mean
        truth = parameters_std * test_y.cpu().numpy() + parameters_mean
    
        # Plot
        corner.corner(pred_samples, truths=truth, labels=['$M_c$', '$q$'])
        plt.show()

**Exercise:** Try learning $(m_1, m_2)$ rather than $(M_c, q)$. The flow should be able to represent the distinct shape.